Python用MetaTrader5のインポート

In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import MetaTrader5 as mt5
import numpy as np
import pandas as pd
import time

ログイン情報を入力

In [3]:
login_ID = 75003107 # ご自身のログインIDを入力
login_server = 'XMTrading-MT5 3' # ご自身のログインサーバーを入力
login_password = 'Takasaki0419' # ご自身のログインパスワードを入力

MT5に接続

In [4]:
# ログイン情報で指定した取引口座でMetaTrader5に接続
if not mt5.initialize(login=login_ID, server=login_server,password=login_password):
    print("initialize() failed, error code =",mt5.last_error())
    quit()

Input設定用のクラスを宣言

In [5]:
class FirstInputData:
  def __init__(self, in_symbol, in_first_lot): #in_symbolに取引対象[string],in_first_lotに取引ロット数[float]
    self.symbol = in_symbol # 取引対象
    self.first_lot = in_first_lot # 初期ロット数
    self.tp_factor = 2.0 # 利確係数
    self.sl_factor = 2.5 # ストップロス係数
    self.magic_number = 10001 # マジックナンバー 
    self.slippage = 10 # スリッページ
    self.bar_period = 30 # 取得する四本値の本数

各種情報取得

In [6]:
def return_point_function(symbol):
  return mt5.symbol_info(symbol).point # 価格の最小単位

テクニカル指標の関数定義

In [7]:
# ATR
def calculate_atr(data, period=14):
    data['tr'] = np.maximum((data['high'] - data['low']), 
                            np.maximum(abs(data['high'] - data['close'].shift()), abs(data['low'] - data['close'].shift())))
    data['atr'] = data['tr'].rolling(window=period).mean()
    return data

# MA
def calculate_ma(data, period=15, price_type='close'):
    data['ma'] = data[price_type].rolling(window=period).mean()
    return data

# RSI
def calculate_rsi(data, period=14, price_type='close'):
    delta = data[price_type].diff()
    gains = delta.where(delta > 0, 0)
    losses = -delta.where(delta < 0, 0)
    avg_gain = gains.rolling(window=period).mean()
    avg_loss = losses.rolling(window=period).mean()
    rs = avg_gain / avg_loss
    data['rsi'] = 100 - (100 / (1 + rs))
    return data

売買判断フラグを設定する関数

In [8]:
def generate_trade_flgs(data, symbol_tick):

    # テクニカル指標の計算
    data = calculate_ma(data)
    data = calculate_rsi(data)

    # 売買判断フラグ
    buy_flg = (symbol_tick.ask > data['ma'].iloc[-2]) & (data['rsi'].iloc[-2] < 40)
    sell_flg = (symbol_tick.bid < data['ma'].iloc[-2]) & (data['rsi'].iloc[-2] > 60)

    return buy_flg, sell_flg

トレードする部分を関数化

In [9]:
def trading_function(symbol, bar_period, tp_factor, sl_factor, first_lot, slippage, magic_number):
    symbol_tick = mt5.symbol_info_tick(symbol)

    # 1分足データを取得
    rates = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_M1, 0, bar_period)#TIMEFRAMEで時間足を変更できる
    data = pd.DataFrame(rates)
    data['time'] = pd.to_datetime(data['time'], unit='s')

    # ATRを計算
    data = calculate_atr(data)
    atr = data['atr'].iloc[-2]

    # 売買判断フラグを設定
    buy_flg, sell_flg = generate_trade_flgs(data, symbol_tick)

    # ポジション情報を取得
    buy_position = None
    sell_position = None
    positions = mt5.positions_get(group='*' + symbol + '*')

    for position in positions:
        order_type = position[5]

        if order_type == 0:
            buy_position = position
        elif order_type == 1:
            sell_position = position


    # buyエントリー
    if buy_flg and buy_position is None:
        tp = round((symbol_tick.ask + atr * tp_factor) / return_point_function(symbol)) * return_point_function(symbol)
        sl = round((symbol_tick.ask - atr * sl_factor) / return_point_function(symbol)) * return_point_function(symbol) - (symbol_tick.ask - symbol_tick.bid)

        request = {
            'symbol': symbol,
            'action': mt5.TRADE_ACTION_DEAL,
            'type': mt5.ORDER_TYPE_BUY,
            'volume': first_lot,
            'price': symbol_tick.ask,
            'sl': sl,
            'tp': tp,
            'deviation': slippage,
            'magic': magic_number,
            'type_time': mt5.ORDER_TIME_GTC,
            'type_filling': mt5.ORDER_FILLING_IOC,
        }

        result = mt5.order_send(request)
    
    # sellエントリー
    if sell_flg and sell_position is None:
        tp = round((symbol_tick.bid - atr * tp_factor) / return_point_function(symbol)) * return_point_function(symbol)
        sl = round((symbol_tick.bid + atr * sl_factor) / return_point_function(symbol)) * return_point_function(symbol) + (symbol_tick.ask - symbol_tick.bid)

        request = {
            'symbol': symbol,
            'action': mt5.TRADE_ACTION_DEAL,
            'type': mt5.ORDER_TYPE_SELL,
            'volume': first_lot,
            'price': symbol_tick.bid,
            'sl': sl,
            'tp': tp,
            'deviation': slippage,
            'magic': magic_number,
            'type_time': mt5.ORDER_TIME_GTC,
            'type_filling': mt5.ORDER_FILLING_IOC,
        }

        result = mt5.order_send(request)


インプット用のインスタンスをオブジェクト化

In [10]:
input1 = FirstInputData("USDJPY", 0.05)
input2 = FirstInputData("EURUSD", 0.05)
input3 = FirstInputData("EURJPY", 0.05)

In [11]:
while True:
    #こっちはドル円用
    #symbol, bar_period, tp_factor, sl_factor, first_lot, slippage, magic_number
    #ドル円用
    trading_function(input1.symbol, input1.bar_period, input1.tp_factor, input1.sl_factor, input1.first_lot, input1.slippage, input1.magic_number)
    #ユーロドル用
    trading_function(input2.symbol, input2.bar_period, input2.tp_factor, input2.sl_factor, input2.first_lot, input2.slippage, input2.magic_number)
    #ユーロ円用
    trading_function(input3.symbol, input3.bar_period, input3.tp_factor, input3.sl_factor, input3.first_lot, input3.slippage, input3.magic_number)


    time.sleep(60)  # 1分間隔でループを実行
